In [ ]:
import simtk.openmm.app as app
import simtk.openmm as mm
import simtk.unit as unit
from mdtraj.reporters import HDF5Reporter
from sys import stdout

Cargar el archivo PDB del disco y agregar residuos y átomos restantes:

In [ ]:
pdb =app.PDBFile("dialanine.pdb")

Especificar el campo de fuerza a utilizar para la simulación:

In [ ]:
forcefield =app.ForceField("amber14-all.xml")

Crear una descripción matematica completa del sistema por simular:

In [ ]:
system = forcefield.createSystem(pdb.topology, nonbondedMethod=app.NoCutoff, constraints=app.HBonds)

Crear el integrador

In [ ]:
integrator = mm.LangevinIntegrator(300*unit.kelvin, 1.0/unit.picoseconds, 2.0*unit.femtoseconds)
integrator.setConstraintTolerance(0.00001)

Preparar la simulación

Definir la plataforma (CUDA o CPU):

In [ ]:
platform = mm.Platform.getPlatformByName('CPU')
properties = {}

Crear el objeto simulación:

In [ ]:
simulation = app.Simulation(pdb.topology, system, integrator, platform, properties)

In [ ]:
simulation.context.setPositions(pdb.positions)

Ejecutar la minimización

In [ ]:
print('Minimizing...')
simulation.minimizeEnergy()

Equilibrar x pasos

In [ ]:
simulation.context.setVelocitiesToTemperature(300*unit.kelvin)
print('Equilibrating...')
simulation.step(25000)

Agregar reporteros

In [ ]:
simulation.reporters.append(HDF5Reporter('trajectory.h5', 1000, cell=True, potentialEnergy=False, kineticEnergy=False, temperature=False))
simulation.reporters.append(app.StateDataReporter(stdout, 100000, step=True,
    potentialEnergy=False, temperature=True, progress=True, remainingTime=True,
    speed=True, totalSteps=25000000, separator='\t'))

Ejecutar la producción

In [ ]:
print('Running Production...')
simulation.step(25000000)
print('Done!')

In [ ]:
simulation.reporters[0].close()